In [3]:
import sys
# 直接导入Smote是不行的，必须将路径加入才可以
sys.path.append("C:Users\perma\PycharmProjects\paper")
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [5]:
# 读取数据
column_names = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "class"]
data_path = "C:\\Users\\perma\\PycharmProjects\\paper\\data\\glass.data"
table = pd.read_table(filepath_or_buffer=data_path,
                      sep=",",
                      header=None,
                      index_col=None,
                      names=column_names)
attributes = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]
data = table[attributes]
label = table["class"]
print("length: data, label", len(data), len(label))
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(X=data)
scaled_data[0:3]
minor_class = [5, 6, 7]
major_class = [1, 2, 3, 4]
new_label = []
for l in label:
    if l in major_class:
        new_label.append(0.0)
    else:
        new_label.append(1.0)


length: data, label 214 214


D:\software\anaconda\exe\envs\forpaper\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import train_test_split

In [7]:
knn = KNeighborsClassifier()
dc = DecisionTreeClassifier()
svm = SVC()
lg = LogisticRegression()
nb = GaussianNB()
algorithm = {"knn": knn, "tree": dc, "svm": svm, "lgs": lg, "nb": nb}
alg_params = {"knn": {"n_neighbors": 7}, 
              "tree": {}, 
              "svm": {}, 
              "lgs": {}, 
              "nb": {}}

In [11]:
# 将少数类和多数类分开提取出来
major = []
major_label = []
minor = []
minor_label = []
for label_index in range(len(new_label)):
    if new_label[label_index] == 1.0:
        minor.append(scaled_data[label_index, :])
        minor_label.append(new_label[label_index])
    else:
        major.append(scaled_data[label_index, :])
        major_label.append(new_label[label_index])
# 定义gmean函数
def g_mean(ground_truth, prediction):
    matrix = confusion_matrix(y_true=ground_truth, y_pred=prediction)
    tpr = matrix[1, 1] / (matrix[1, 1] + matrix[1, 0])
    tpn = matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
    return np.sqrt(tpr * tpn)
result = dict([(key, {"gmean": list(), "fscore": list()}) for key in algorithm.keys()])
# 实验次数
n=100
for i in range(n):
    for key in algorithm.keys():
        alg = algorithm[key]
        alg.set_params(**alg_params[key])
        major_train, major_test, \
        major_label_train, major_label_test \
            = train_test_split(major, major_label, shuffle=True, test_size=0.25)
        minor_train, minor_test, \
        minor_label_train, minor_label_test \
            = train_test_split(minor, minor_label, shuffle=True, test_size=0.25)
        data_train = np.concatenate((major_train, minor_train), axis=0)
        label_train = np.concatenate((major_label_train, minor_label_train), axis=0)
        data_test = np.concatenate((major_test, minor_test), axis=0)
        label_test = np.concatenate((major_label_test, minor_label_test), axis=0)
        alg.fit(X=data_train, y=label_train)
        prediction = alg.predict(X=data_test)
        gmean = g_mean(label_test, prediction)
        fscore = f1_score(y_true=label_test, y_pred=prediction) / 2
        result[key]["gmean"].append(gmean)
        result[key]["fscore"].append(fscore)
new_line = 0       
for alg_key in result.keys():
    new_line += 1
    alg = result[alg_key]
    for score_key in alg.keys():
        mean_score = np.mean(alg[score_key])
        print(alg_key, " : ", score_key, " : ", mean_score, end="-----------------")
    print("\n")


knn  :  gmean  :  0.8647075554828458-----------------knn  :  fscore  :  0.40711706657493757-----------------

tree  :  gmean  :  0.899541036179529-----------------tree  :  fscore  :  0.4275839412930431-----------------

svm  :  gmean  :  0.8323414062457237-----------------svm  :  fscore  :  0.39503694708268605-----------------

lgs  :  gmean  :  0.8199082752284845-----------------lgs  :  fscore  :  0.3888931812445542-----------------

nb  :  gmean  :  0.8639920553718017-----------------nb  :  fscore  :  0.40086981398933746-----------------

